In [ ]:
# Imports
import datetime
import os
from itertools import combinations

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tabulate import tabulate
import numpy as np
import pingouin as pg
from scipy.stats import shapiro, levene
import statsmodels.stats.multitest as smm
import scikit_posthocs as sp
from scipy import stats

In [ ]:
def parse_logcat(log_path, verbose:bool=False):
    '''
    Parse the log, trimming useless content from the beginning, and adding the data to dictionaries
    '''
    if verbose:
        print("Parsing logcat file...")

    log = []
    event_log = []

    with open(log_path, "r", encoding="utf-16") as logcat_file:
        logcat_lines = logcat_file.readlines()

    # Trim the file to keep only the lines from the first scene start
    for i in range(len(logcat_lines) - 1):
        if '[END] Scene' in logcat_lines[i] and 'Evaluation script enabled' in logcat_lines[i + 1]:
            # Trim the file to keep only the lines from the found pair onwards
            logcat_lines = logcat_lines[i:]
            if verbose:
                print("File trimmed successfully.")
            
            break

    # Parse the logcat file
    for line in logcat_lines:
        parts = line.split()

        try:
            # Get unix timestamp with milliseconds
            current_year = datetime.datetime.now().year
            timestamp_string =  f"{current_year}-{parts[0]} {parts[1]}"
            timestamp_format = "%Y-%m-%d %H:%M:%S.%f"
            dt = datetime.datetime.strptime(timestamp_string, timestamp_format)
            unix = int(dt.timestamp() * 1000)

            # Get performance stats
            if parts[5] == "VrApi":
                stats = parts[7].split(",")

                # Create a dictionary with the stats
                entry = {
                    "timestamp": unix, 
                    "fps": stats[0].replace("FPS=", "").split("/")[0],
                    "refresh_rate": stats[0].replace("FPS=", "").split("/")[1],
                    "temperature1": stats[17].replace("Temp=", "").replace("C", "").split("/")[0],
                    "temperature2": stats[17].replace("Temp=", "").replace("C", "").split("/")[1],
                    "frame_time": stats[19].replace("App=", "").replace("ms", ""),
                    "stale_frames": stats[4].replace("Stale=", ""),
                    "free_memory": stats[15].replace("Free=", "").replace("MB", ""),
                    "total_render_time": stats[21].replace("CPU&GPU=", "").replace("ms", ""),
                    "gpu": stats[24].replace("GPU%=", ""),
                    "cpu_average": stats[25].replace("CPU%=", "").split("(W")[0],
                    "cpu_worst": stats[25].replace("CPU%=", "").split("(W")[1].replace(")","")
                }

                log.append(entry)

            # Get events
            elif parts[5] == "Unity" and parts[4] == "I":
                if "[START]" == parts[7] or "[END]" == parts[7]:
                    event = {"timestamp": unix, "state": parts[7], "event": " ".join(parts[8:])}
                    event_log.append(event)

        except (IndexError, ValueError):
            continue

    return log, event_log

In [ ]:
def adjust_time(log, event_log, decimals: int=2, verbose:bool=False):
    '''
    Adjust the timestamps in the log to be relative to the first entry.
    Filter out entries with negative timestamps.
    '''
    if verbose:
        print("Adjusting timestamps to be relative to the first event...")

    first_timestamp = event_log[0]["timestamp"]

    # Adjust timestamps to be relative to the first event (scene load)
    for entry in log:
        entry["timestamp"] = round((entry["timestamp"] - first_timestamp) / 1000, decimals)

    for entry in event_log:
        entry["timestamp"] = round((entry["timestamp"] - first_timestamp) / 1000, decimals)
        
    # Filter out performance log entries with negative timestamps
    filtered_log = [entry for entry in log if entry["timestamp"] >= 0]

    if verbose:
        print("Timestamps adjusted successfully.")
        print(filtered_log)
        print(event_log)

    return filtered_log, event_log

### Both ends of the measurements need to be trimmed as they have additional performance strain from feature initializations and user actions.

In [ ]:
def trim_logs(log, event_log, verbose:bool=False):
    '''
    Trim 10 seconds from the start and end of the logs.
    '''
    if verbose:
        print("Trimming logs...")

    # Filter events that contain the substring "test"
    test_events = [event for event in event_log if "test" in event["event"]]

    # Split the logs 10 seconds before the first test event and 80 seconds after the last test event
    first_test_event = test_events[0]["timestamp"]
    last_test_event = test_events[-1]["timestamp"]

    trimmed_log = [entry for entry in log if entry["timestamp"] >= first_test_event - 15 and entry["timestamp"] <= last_test_event + 30]

    if verbose:
        print(f"First test event: {first_test_event}")
        print(f"Last test event: {last_test_event}")

    return trimmed_log, event_log

In [ ]:
def visualize_performance(log, event_log, mode: str, verbose:bool=False):
    '''
    Visualize frame time over time.
    '''
    if verbose:
        print("Visualizing performance...")

    # Draw frame time over time
    frame_times = [float(entry["frame_time"]) for entry in log]
    timestamps = [entry["timestamp"] for entry in log]

    plt.figure(figsize=(20, 6))
    plt.plot(timestamps, frame_times)
    
    added_labels = set()
    loading_start = {}
    max_timestamp = 0
    event_colors = {
        "Scene": "blue",
        "CO2": "green",
        "Fade": "orange",
        "Barchart": "purple",
        "thermostat test": "red",
        "CO2 test": "green",
        "barchart test": "purple",
        "All tests": "blue"
    }
    test_times = []

    # Draw vertical lines for events
    for event in event_log:
        event_type = event.get("event", "default")
        event_state = event.get("state", "")
        
        if event_type in ["thermostat test", "CO2 test", "barchart test", "All tests"]:
            # Default to black if event type is not in event_colors
            color = event_colors.get(event_type, "black")

            if event_type not in added_labels:
                plt.axvline(x=event["timestamp"], color=color, linestyle='--', label=event_type, zorder=1)
                added_labels.add(event_type)
            else:
                plt.axvline(x=event["timestamp"], color=color, linestyle='--', zorder=1)
                
            if event_state == "[START]":
                loading_start[event["event"]] = event["timestamp"]
            elif event_state == "[END]":
                try:
                    plt.axvspan(loading_start[event["event"]], event["timestamp"], color="gray", alpha=0.2)
                    test_times.append((loading_start[event["event"]], event["timestamp"]))

                    if verbose:
                        print(f"Time for {event["event"]}: {event['timestamp'] - loading_start[event["event"]]}")
                        print(f"    Starting time: {loading_start[event["event"]]}, Ending time: {event['timestamp']}")
                except (NameError, TypeError) as e:
                    print(e)
                
            continue

        # Update max_timestamp
        if event["timestamp"] > max_timestamp:
            max_timestamp = event["timestamp"]

    plt.legend()
    plt.xlabel("Time (s)")
    plt.ylabel("Frame time (ms)")
    plt.title("Frame time over time in " + mode)
    plt.show()

    return test_times

# Preprocessing for analysis

In [ ]:
def preprocess_event_log(event_log: list, num_all_tests: int = 1, ignored_time: float = 0.0, verbose:bool=False):
    '''
    Preprocess the event log to include idle time and combine simultaneous tests into one.
    '''
    if verbose:
        print("Preprocessing event log...")

    tests = ["thermostat test", "CO2 test", "barchart test"]
    processed_log = []
    last_end = None
    all_test_events = []
    
    # Filter event log to only include the tests
    event_log = [event for event in event_log if event["event"] in tests]

    # Loop over tests before simultaneous tests
    for event in event_log[:-6*num_all_tests]:
        if event["event"] in tests:

            if len(processed_log) == 0:
                processed_log.append({'timestamp': ignored_time, 'state': '[START]', 'event': 'Idle'})
                processed_log.append({'timestamp': event["timestamp"] - 0.01, 'state': '[END]', 'event': 'Idle'})

            if event["state"] == "[END]":
                last_end = event["timestamp"]
            elif event["state"] == "[START]":
                if last_end is not None:
                    processed_log.append({'timestamp': last_end + 0.01, 'state': '[START]', 'event': 'Idle'})
                    processed_log.append({'timestamp': event["timestamp"] - 0.01, 'state': '[END]', 'event': 'Idle'})
            processed_log.append(event)
    
    try:
        processed_log.append({'timestamp': last_end + 0.01, 'state': '[START]', 'event': 'Idle'})
    except TypeError as e:
        if verbose:
            print(e)
            print(last_end)
    
    # Handle simultaneous tests
    thermostat_test_events = [event for event in event_log if event["event"] == "thermostat test"]
    co2_test_events = [event for event in event_log if event["event"] == "CO2 test"]
    barchart_test_events = [event for event in event_log if event["event"] == "barchart test"]

    for i in range(num_all_tests):
        # Compare the times to get the earliest end time
        earliest_end = min(thermostat_test_events[-1]["timestamp"], co2_test_events[-1]["timestamp"], barchart_test_events[-1]["timestamp"])
        latest_end = max(thermostat_test_events[-1]["timestamp"], co2_test_events[-1]["timestamp"], barchart_test_events[-1]["timestamp"])

        earliest_start = min(thermostat_test_events[-2]["timestamp"], co2_test_events[-2]["timestamp"], barchart_test_events[-2]["timestamp"])
        latest_start = max(thermostat_test_events[-2]["timestamp"], co2_test_events[-2]["timestamp"], barchart_test_events[-2]["timestamp"])
        
        all_test_events.append({"timestamp": earliest_start - 0.01, "state": "[END]", "event": "Idle"})
        all_test_events.append({"timestamp": latest_start, "state": "[START]", "event": "All tests"})
        all_test_events.append({"timestamp": earliest_end, "state": "[END]", "event": "All tests"})
        all_test_events.append({"timestamp": latest_end + 0.01, "state": "[START]", "event": "Idle"})
        all_test_events.append({'timestamp': latest_end + 60, 'state': '[END]', 'event': 'Idle'})

        thermostat_test_events = thermostat_test_events[:-2]
        co2_test_events = co2_test_events[:-2]
        barchart_test_events = barchart_test_events[:-2]

    processed_log.extend(all_test_events)

    return processed_log

In [ ]:
def bin_data(log, processed_event_log, verbose:bool=False):
    '''
    Binning data based on the event log.
    '''
    if verbose:
        print("Binning data...")

    binned_data = {
        "Idle": [],
        "thermostat test": [],
        "CO2 test": [],
        "barchart test": [],
        "All tests": []
    }

    for i in range(int(len(processed_event_log) / 2)):
        start = processed_event_log[2*i]
        end = processed_event_log[2*i + 1]

        # Filter log entries between start and end
        binned_data[start["event"]].extend([entry for entry in log if start["timestamp"] <= entry["timestamp"] <= end["timestamp"]])

    # Print all key value pairs on separate lines
    for key, value in binned_data.items():
        if verbose:
            print(f"{key}: {value}")

    return binned_data

### Stale frame analysis was only for testing purposes; was not used in the final analysis of the thesis.

In [ ]:
def calculate_stale_frames(binned_data):
    '''
    Calculate the total number of stale frames in each bin.
    '''
    stale_frames = {}

    for key, value in binned_data.items():
        total_stale_frames = sum([int(entry["stale_frames"]) for entry in value])
        stale_frames[key] = total_stale_frames

    return stale_frames

In [ ]:
def visualize_stale_frames(binned_data, mode: str):
    '''
    Visualize stale frames in idle bin.
    '''
    idle_data = binned_data["Idle"]
    timestamps = [entry["timestamp"] for entry in idle_data]
    stale_frames = [int(entry["stale_frames"]) for entry in idle_data]

    plt.figure(figsize=(20, 6))
    plt.plot(timestamps, stale_frames)
    plt.xlabel("Time (s)")
    plt.ylabel("Stale frames")
    plt.title("Stale frames in idle state in " + mode)
    plt.show()

### Correlation matrix and descriptive stats

In [ ]:
def create_correlation_matrix(log, mode: str, ignored_keys: list = [], verbose:bool=False):
    '''
    Create correlation matrix from performance data.
    '''
    if verbose:
        print("Creating correlation matrix...")
        
    # Remove ignored keys
    for key in ignored_keys:
        log = [{k: v for k, v in entry.items() if k != key} for entry in log]

    df = pd.DataFrame(log)
    corr = df.corr()

    # Visualize using seaborn
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title("Correlation matrix of performance metrics in " + mode)
    plt.show()

    return corr

In [ ]:
def descriptive_statistics(log, ignored_keys: list = [], decimals: int = 2, verbose:bool=False):
    '''
    Calculate descriptive statistics for performance data.
    
    Args:
        log: List of dictionaries with performance data
        ignored_keys: List of keys to ignore when calculating statistics
        decimals: Number of decimals to round the statistics to
        verbose: Print information about the process
    '''
    if verbose:
        print("Calculating descriptive statistics...")
        
    statistics = {}
    
    # Remove ignored keys
    for key in ignored_keys:
        log = [{k: v for k, v in entry.items() if k != key} for entry in log]

    # Convert to numpy structured array
    dtype = [(key, float) for key in log[0].keys()]
    data = np.array([tuple(entry.values()) for entry in log], dtype=dtype)

    # Calculate statistics for each metric
    for key in data.dtype.names:
        statistics[key] = {
            'average': np.mean(data[key]),
            'median': np.median(data[key]),
            'std_dev': np.std(data[key]),
            'min': np.min(data[key]),
            'max': np.max(data[key])
        }

    # Round values
    for key, value in statistics.items():
        for metric, val in value.items():
            statistics[key][metric] = np.round(val, decimals)

    # Print as a table with original keys on the horizontal axis and nested keys on the vertical axis
    headers = ["Statistic"] + list(statistics.keys())
    table_data = []
    
    for stat_name in statistics[next(iter(statistics))].keys():
        row = [stat_name] + [statistics[key][stat_name] for key in statistics.keys()]
        table_data.append(row)

    print(tabulate(table_data, headers=headers, tablefmt="fancy_grid"))

    # Also print the results ready for overleaf
    for row in table_data:
        print(" & ".join(map(str, row)))
    
    return

### Combination of measurements

- Find start and end points of events
- Set latest start and earliest end and event time range
- earliest start and latest end as idle range

- calculate average of data
- calculate std dev of data

In [ ]:
def aggregate_measurements(performance_logs:list, event_logs:list, verbose:bool=False):
    '''
    Aggregate performance measurements from multiple logs.
    '''
    if verbose:
        print("Aggregating performance measurements...")

    # Aggregate event logs
    preprocessed_event_logs = []
    aggregated_event_log = []

    for log in event_logs:
        preprocessed_event_log = preprocess_event_log(log)
        preprocessed_event_logs.append(preprocessed_event_log)

    # Loop over all events
    for i in range(len(preprocessed_event_logs[0])):
        comparable_events = [event_log[i] for event_log in preprocessed_event_logs]

        # Handle idle events
        if all(event["event"] == "Idle" for event in comparable_events) or all("test" in event["event"] for event in comparable_events):
            if comparable_events[0]["state"] == "[START]":
                # Append the latest start event
                aggregated_event_log.append(max(comparable_events, key=lambda x: x["timestamp"]))
            elif comparable_events[0]["state"] == "[END]":
                # Append the earliest end event
                aggregated_event_log.append(min(comparable_events, key=lambda x: x["timestamp"]))
        
    if verbose:
        print("Aggregated event log and its length:")
        print(aggregated_event_log)
        print(len(aggregated_event_log))

        print("Lengths of given performance logs:")
        print([len(log) for log in performance_logs])

    # Combine performance logs by calculating the average of each metric
    aggregated_performance_log = []

    # Loop over range of the longest log
    for i in range(max([len(log) for log in performance_logs])):
        # Get all entries at index i
        entries = [log[i] for log in performance_logs if i < len(log)]
        # Calculate the average value of each metric across all logs at each index
        aggregated_entry = {
            key: sum([float(entry[key]) for entry in entries]) / len(entries) for key in entries[0].keys()
        }
        aggregated_performance_log.append(aggregated_entry)

    if verbose:
        print("Aggregated performance log:")
        print(aggregated_performance_log)

    return aggregated_performance_log, aggregated_event_log

    

def process_logs(log_paths:list, verbose:bool=False):
    '''
    Process logs from the given paths
    '''
    performance_logs = []
    event_logs = []

    for path in log_paths:
        log, event_log = parse_logcat(path, verbose)
        log, event_log = adjust_time(log, event_log, decimals=0, verbose=verbose)
        log, event_log = trim_logs(log, event_log, verbose)

        performance_logs.append(log)
        event_logs.append(event_log)

    return performance_logs, event_logs

In [ ]:
def export_binned_data_csv(data:dict, mode:str):
    '''
    Combine and export binned data into a single CSV file.
    '''
    combined_data = []
    for key, value in data.items():
        for entry in value:
            entry["state"] = key.lower().replace(" ", "_")
            combined_data.append(entry)

    df = pd.DataFrame(combined_data)
    df.to_csv(f"./output/{mode.lower().replace(" ", "_")}.csv", index=False)
    
def clear_export():
    '''
    Clear the existing CSV file.
    '''
    with open("./output/output.csv", "w") as f:
        f.write("")

In [ ]:
def export_all():
    '''
    Export all data to a single CSV file.
    '''
    output_df = pd.DataFrame()
    # Loop over files in ./input
    for file in os.listdir("./input"):
        if file.endswith(".log"):
            # Get file name
            filename = file.replace(".log", "").replace("logcat_", "")
            filename_list = filename.split("_")
            
            # Get metadata
            mode = filename_list.pop(0).upper()
            run = filename_list.pop(-1)
            movement = "liike" in filename_list

            # Process the log
            performance_logs, event_logs = process_logs([f"./input/{file}"])
            print(performance_logs)

            for entry in performance_logs[0]:
                entry['mode'] = mode
                entry['movement'] = movement
                entry['condition'] = mode + ("_Movement" if movement else "_NoMovement")
                entry['run'] = run

            combined_data = []
            for datapoint in performance_logs[0]:
                combined_data.append(datapoint)

            processed_df = pd.DataFrame(combined_data)
            # Append to output df
            output_df = pd.concat([output_df, processed_df], ignore_index=True)

    # add index column name
    output_df.index.name = "index"
    # start index from 1
    output_df.index = output_df.index + 1
    output_df.to_csv(f"./output/output.csv", mode='w', header=True, index=True)

    return

    


# clear_export()
export_all()


In [ ]:
def analysis_of_variance(metric:str, correction:bool):
    '''
    Performs analysis of variance.
    Checks if data is normally distributed, if conditions have similar variance and if sphericity is met.
    Performs ANOVA and Friedman test.
    If checks fail, Friedman test should be used.
    '''
    print("-----")
    print(metric.upper())
    # Load data CSV
    data = pd.read_csv("./output/output.csv", header=0)
    # Calculate logarithm of frame time
    data["log_" + metric] = np.log(data[metric])

    print("-----")
    print("TESTS")
    print("-----")

    # Check if data is normally distributed
    for condition in data["condition"].unique():
        _, p = shapiro(data[data["condition"] == condition][metric])

        if p < 0.05:
            print(f"{condition} is not normally distributed")
        else:
            print(f"{condition} is normally distributed")
        # print(p)

    # Check if conditions have similar variance
    samples = [data[data["condition"] == condition][metric] for condition in data["condition"].unique()]
    _, p = levene(*samples)

    if p < 0.05:
        print("Conditions do not have similar variance")
    else:
        print("Conditions have similar variance")

    # # Check for sphericity
    sphericity = pg.sphericity(data, dv=metric, subject="run", within="condition")
    print("Sphericity: ", sphericity[0])

    print("-----")
    print("ANOVA")
    print("-----")
    data['mode'] = data['mode'].astype('category')
    anova = pg.rm_anova(data=data, dv=metric, within="condition", subject='run', correction=correction, detailed=True)

    for row in anova.iterrows():
        if row[1]["p-unc"] > 0 and row[1]["p-unc"] < 0.001:
            anova.at[row[0], "p-unc"] = "<0.001"
        elif row[1]["p-unc"] > 0.001 and row[1]["p-unc"] < 0.01:
            anova.at[row[0], "p-unc"] = "<0.01"
        elif row[1]["p-unc"] > 0.01:
            anova.at[row[0], "p-unc"] = round(row[1]["p-unc"], 3)

    # Remove unwanted columns
    anova = anova.drop(columns=["p-GG-corr", "ng2", "eps", "sphericity", "W-spher", "p-spher"])

    # Round values
    anova = anova.round(3)
    
    print(anova)

    print("-----")
    print("T-TEST")
    print("-----")

    results = []
    t_results = []

    # Loop over all combinations of conditions
    for c1, c2 in combinations(data["condition"].unique(), 2):
        # Get the data for the two conditions
        x = data[data["condition"] == c1][metric]
        y = data[data["condition"] == c2][metric]

        # t-test
        t_result = pg.ttest(x, y, correction="auto", paired=False)
        t_row = [c1, c2] + t_result.iloc[0].tolist()
        t_results.append(t_row)

    # Combine results into a dataframe
    df_t = pd.DataFrame(t_results, columns=["Condition 1", "Condition 2"] + list(t_result.columns))
    
    # remove alternative column from df
    df_t = df_t.drop(columns=["alternative"])

    df_t["Condition 1"] = df_t["Condition 1"].str.replace("_", " ")
    df_t["Condition 2"] = df_t["Condition 2"].str.replace("_", " ")

    # Loop through p-values and format them
    for i in range(len(df_t)):
        if df_t.iloc[i]["p-val"] > 0 and df_t.iloc[i]["p-val"] < 0.001:
            df_t.at[i, "p-val"] = "<0.001"
        elif df_t.iloc[i]["p-val"] > 0.001 and df_t.iloc[i]["p-val"] < 0.01:
            df_t.at[i, "p-val"] = "<0.01"
        elif df_t.iloc[i]["p-val"] > 0.01:
            df_t.at[i, "p-val"] = round(df_t.iloc[i]["p-val"], 3)

    # Round to 2 decimals
    df_t = df_t.round(2)

    # Filter out dof and bf10 columns from df
    df_t = df_t.drop(columns=["dof", "BF10"])

    print(tabulate(df_t, headers='keys', tablefmt='pretty'))
    print("        \\toprule")
    print(r"        Condition 1 & Condition 2 & T & p-value & CI95\% & Cohen's d & Power \\\\")
    print("        \\midrule")
    for _, row in df_t.iterrows():
        print("        " + " & ".join(map(str, row)) + " \\\\")
    print("        \\bottomrule")

    a = pg.pairwise_tests(data, dv=metric, within="condition", subject="run", padjust='bonf', parametric=False, alpha=0.05, correction='auto')
    print(a)

analysis_of_variance("frame_time", True)
analysis_of_variance("fps", True)
analysis_of_variance("gpu", True)
analysis_of_variance("cpu_average", True)
analysis_of_variance("cpu_worst", True)
analysis_of_variance("total_render_time", True)
analysis_of_variance("stale_frames", True)
analysis_of_variance("free_memory", True)

# Run analysis on batches

In [ ]:
paths_vr = [r".\logcat_final\logcat_vr_1.log", r".\logcat_final\logcat_vr_2.log", r".\logcat_final\logcat_vr_3.log", r".\logcat_final\logcat_vr_4.log", r".\logcat_final\logcat_vr_5.log"]
paths_vr_liike = [r".\logcat_final\logcat_vr_liike_1.log", r".\logcat_final\logcat_vr_liike_2.log", r".\logcat_final\logcat_vr_liike_3.log", r".\logcat_final\logcat_vr_liike_4.log", r".\logcat_final\logcat_vr_liike_5.log"]
paths_vr_pt = [r".\logcat_final\logcat_vr-pt_1.log", r".\logcat_final\logcat_vr-pt_2.log", r".\logcat_final\logcat_vr-pt_3.log", r".\logcat_final\logcat_vr-pt_4.log", r".\logcat_final\logcat_vr-pt_5.log"]
paths_vr_pt_liike = [r".\logcat_final\logcat_vr-pt_liike_1.log", r".\logcat_final\logcat_vr-pt_liike_2.log", r".\logcat_final\logcat_vr-pt_liike_3.log", r".\logcat_final\logcat_vr-pt_liike_4.log", r".\logcat_final\logcat_vr-pt_liike_5.log"]
paths_ar = [r".\logcat_final\logcat_ar_1.log", r".\logcat_final\logcat_ar_2.log", r".\logcat_final\logcat_ar_3.log", r".\logcat_final\logcat_ar_4.log", r".\logcat_final\logcat_ar_5.log"]
paths_ar_liike = [r".\logcat_final\logcat_ar_liike_1.log", r".\logcat_final\logcat_ar_liike_2.log", r".\logcat_final\logcat_ar_liike_3.log", r".\logcat_final\logcat_ar_liike_4.log", r".\logcat_final\logcat_ar_liike_5.log"]

def analyze_batch(paths:list, mode:str, verbose:bool=False):
    '''
    Analyze a batch of logs.

    Args:
        paths: list of paths to the log files
        mode: string representing the conditions of the measurements
        verbose: boolean for printing debug information
    '''

    performance_logs, event_logs = process_logs(paths, verbose)
    performance_logs, event_logs = aggregate_measurements(performance_logs, event_logs, verbose)
    binned_data = bin_data(performance_logs, event_logs, verbose)
    # Loop through binned data and print the length of each bin
    for key, value in binned_data.items():
        print(f"{key}: {len(value)}")
    export_binned_data_csv(binned_data, mode)

    visualize_performance(performance_logs, event_logs, mode, verbose)

    corr = create_correlation_matrix(performance_logs, mode, ignored_keys, verbose)
    print(corr)

    # stale_frames = calculate_stale_frames(binned_data)
    # visualize_stale_frames(binned_data, mode)

    print("Descriptive statistics for all logs:")
    descriptive_statistics(performance_logs, ignored_keys, decimals=2, verbose=verbose)

    print("Descriptive statistics for idle:")
    descriptive_statistics(binned_data["Idle"], ignored_keys, decimals=2, verbose=verbose)
    print("Descriptive statistics for thermostat test:")
    descriptive_statistics(binned_data["thermostat test"], ignored_keys, decimals=2, verbose=verbose)
    print("Descriptive statistics for CO2 test:")
    descriptive_statistics(binned_data["CO2 test"], ignored_keys, decimals=2, verbose=verbose)
    print("Descriptive statistics for barchart test:")
    descriptive_statistics(binned_data["barchart test"], ignored_keys, decimals=2, verbose=verbose)
    print("Descriptive statistics for simultaneous tests:")
    descriptive_statistics(binned_data["All tests"], ignored_keys, decimals=2, verbose=verbose)

ignored_keys = ["timestamp", "refresh_rate", "temperature1", "temperature2", "state"]

analyze_batch(paths_vr, "VR", verbose=False)
analyze_batch(paths_vr_liike, "VR with movement", verbose=False)
analyze_batch(paths_vr_pt, "VR with passthrough", verbose=False)
analyze_batch(paths_vr_pt_liike, "VR with passthrough and movement", verbose=False)
analyze_batch(paths_ar, "AR", verbose=False)
analyze_batch(paths_ar_liike, "AR with movement", verbose=False)

In [ ]:
def visualize_performances_together(logs:list, event_log, mode: str, verbose:bool=False):
    '''
    Visualize frame time over time
    '''
    if verbose:
        print("Visualizing performance...")

    modes = ["VR", "VR-PT", "AR"]
    plt.figure(figsize=(20, 6))
    plt.ylim(bottom=0, top=25)
    plt.yticks(np.arange(0, 26, 2.5))
    plt.grid(axis='y', linestyle='--', alpha=0.5)

    for i, log in enumerate(logs):
        frame_times = [float(entry["frame_time"]) for entry in log]
        timestamps = [entry["timestamp"] for entry in log]
        plt.plot(timestamps, frame_times, label=f"{modes[i]}")
    
    added_labels = set()
    loading_start = {}
    event_colors = {
        "Scene": "blue",
        "CO2": "green",
        "Fade": "orange",
        "Barchart": "purple",
        "thermostat test": "red",
        "CO2 test": "green",
        "barchart test": "purple",
        "All tests": "blue"
    }
    test_times = []

    # Draw vertical lines for events
    for event in event_log:
        event_type = event.get("event", "default")
        event_state = event.get("state", "")
        
        if event_type in ["thermostat test", "CO2 test", "barchart test", "All tests"]:
            # Default to black if event type is not in event_colors
            color = event_colors.get(event_type, "black")

            if event_type not in added_labels:
                plt.axvline(x=event["timestamp"], color=color, linestyle='--', label=event_type, zorder=1)
                added_labels.add(event_type)
            else:
                plt.axvline(x=event["timestamp"], color=color, linestyle='--', zorder=1)
                
            if event_state == "[START]":
                loading_start[event["event"]] = event["timestamp"]
            elif event_state == "[END]":
                try:
                    plt.axvspan(loading_start[event["event"]], event["timestamp"], color="gray", alpha=0.2)
                    test_times.append((loading_start[event["event"]], event["timestamp"]))

                    if verbose:
                        print(f"Time for {event["event"]}: {event['timestamp'] - loading_start[event["event"]]}")
                        print(f"    Starting time: {loading_start[event["event"]]}, Ending time: {event['timestamp']}")
                except (NameError, TypeError) as e:
                    print(e)
                
            continue

    plt.legend()
    plt.xlabel("Time (s)")
    plt.ylabel("Frame time (ms)")
    plt.title("Frame time over time including " + mode)
    plt.show()

visualize_performances_together([performance_logs_vr, performance_logs_vr_pt, performance_logs_ar], event_logs_vr, "all modes without movement", verbose=False)
visualize_performances_together([performance_logs_vr_liike, performance_logs_vr_pt_liike, performance_logs_ar_liike], event_logs_vr_liike, "all modes with movement", verbose=False)

# Run analysis on single emasurements

In [ ]:
pre_vr = r".\pre\logcat_vr_liike.log"
pre_vr_2 = r".\pre\vr-testi.log"

vr_1 = r".\logcat_final\logcat_vr_1.log"
vr_2 = r".\logcat_final\logcat_vr_2.log"
vr_3 = r".\logcat_final\logcat_vr_3.log"
vr_4 = r".\logcat_final\logcat_vr_4.log"
vr_5 = r".\logcat_final\logcat_vr_5.log"

vr_liike_1 = r".\logcat_final\logcat_vr_liike_1.log"
vr_liike_2 = r".\logcat_final\logcat_vr_liike_2.log"
vr_liike_3 = r".\logcat_final\logcat_vr_liike_3.log"
vr_liike_4 = r".\logcat_final\logcat_vr_liike_4.log"
vr_liike_5 = r".\logcat_final\logcat_vr_liike_5.log"

faulty_vr_liike_4 = r".\logcat_final\faulty\logcat_vr_liike_4.log"

vr_pt_1 = r".\logcat_final\logcat_vr_pt_1.log"
vr_pt_2 = r".\logcat_final\logcat_vr_pt_2.log"
vr_pt_3 = r".\logcat_final\logcat_vr_pt_3.log"
vr_pt_4 = r".\logcat_final\logcat_vr_pt_4.log"
vr_pt_5 = r".\logcat_final\logcat_vr_pt_5.log"

vr_pt_liike_1 = r".\logcat_final\logcat_vr_pt_liike_1.log"
vr_pt_liike_2 = r".\logcat_final\logcat_vr_pt_liike_2.log"
vr_pt_liike_3 = r".\logcat_final\logcat_vr_pt_liike_3.log"
vr_pt_liike_4 = r".\logcat_final\logcat_vr_pt_liike_4.log"
vr_pt_liike_5 = r".\logcat_final\logcat_vr_pt_liike_5.log"

ar_1 = r".\logcat_final\logcat_ar_1.log"
ar_2 = r".\logcat_final\logcat_ar_2.log"
ar_3 = r".\logcat_final\logcat_ar_3.log"
ar_4 = r".\logcat_final\logcat_ar_4.log"
ar_5 = r".\logcat_final\logcat_ar_5.log"

ar_liike_1 = r".\logcat_final\logcat_ar_liike_1.log"
ar_liike_2 = r".\logcat_final\logcat_ar_liike_2.log"
ar_liike_3 = r".\logcat_final\logcat_ar_liike_3.log"
ar_liike_4 = r".\logcat_final\logcat_ar_liike_4.log"
ar_liike_5 = r".\logcat_final\logcat_ar_liike_5.log"


test_times = []

def analyze(path:str, mode:str):
    logg, event_logg = parse_logcat(path)
    # print_log(logg, event_logg)
    logg, event_logg = adjust_time(logg, event_logg, decimals=0)
    # print_log(logg, event_logg)
    logg, event_logg = trim_logs(logg, event_logg)
    # print_log(logg, event_logg)

    # test_times.append(visualize_performance(logg, event_logg, mode))

    processed_event_logg = preprocess_event_log(event_logg)

    binned_data = bin_data(logg, processed_event_logg)

    # stale_frames = calculate_stale_frames(binned_data)

    # print(stale_frames)

    visualize_stale_frames(binned_data, "VR")

    corr = create_correlation_matrix(logg, "VR", ignored_keys)

    descriptive_statistics(logg, ignored_keys)
    print(logg)
    descriptive_statistics(binned_data["All tests"], ignored_keys)
    descriptive_statistics(binned_data["Idle"], ignored_keys)

    return

# analyze(faulty_vr_liike_4)
# analyze(pre_vr_2, '')

analyze(vr_1, 'VR with no movement')
analyze(vr_2, 'VR with no movement')
analyze(vr_3, 'VR with no movement')
analyze(vr_4, 'VR with no movement')
analyze(vr_5, 'VR with no movement')

# analyze(vr_liike_1, 'VR with movement')
# analyze(vr_liike_2, 'VR with movement')
# analyze(vr_liike_3, 'VR with movement')
# analyze(vr_liike_4, 'VR with movement')
# analyze(vr_liike_5, 'VR with movement')

# analyze(vr_pt_1, 'VR with no movement')
# analyze(vr_pt_2, 'VR with no movement')
# analyze(vr_pt_3, 'VR with no movement')
# analyze(vr_pt_4, 'VR with no movement')
# analyze(vr_pt_5, 'VR with no movement')

# analyze(vr_pt_liike_1, 'VR with movement')
# analyze(vr_pt_liike_2, 'VR with movement')
# analyze(vr_pt_liike_3, 'VR with movement')
# analyze(vr_pt_liike_4, 'VR with movement')
# analyze(vr_pt_liike_5, 'VR with movement')

# analyze(ar_1, 'AR with no movement')
# analyze(ar_2, 'AR with no movement')
# analyze(ar_3, 'AR with no movement')
# analyze(ar_4, 'AR with no movement')
# analyze(ar_5, 'AR with no movement')

# analyze(ar_liike_1, 'AR with movement')
# analyze(ar_liike_2, 'AR with movement')
# analyze(ar_liike_3, 'AR with movement')
# analyze(ar_liike_4, 'AR with movement')
# analyze(ar_liike_5, 'AR with movement')

# for test_time in test_times:
#     print(test_time)
